In [ ]:
import os
import datetime

import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.np_utils import to_categorical   
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras_applications import vgg16, vgg19, inception_v3, resnet50, mobilenet, mobilenet_v2, inception_resnet_v2, xception, densenet, nasnet
from keras.callbacks import TensorBoard
import numpy as np
import pandas as pd
from livelossplot import PlotLossesKeras

In [ ]:
def generate_bottlenecks_and_labels(model, image_shape, augment_factor, flow_kwargs={}):
    '''
    Self Explanatory
    
    Arguments
    - model: keras.model.Model, usually a pre-existing model excluding top-layers,
             with pre-trained weights
    - image_shape: tuple(x, y)
    - augment_factor: how many times to augment non-test images
    
    Returns a tuple of 2 elements
    - bottlenecks: dict of bottleneck np.ndarrays, by dataset
    - labels: dict of label np.ndarrays, by dataset
    '''
    
    # define augmenations
    transform_parameters = {
        'zx': 0.6,
        'zy': 0.6,
    }
    zoom_gen = ImageDataGenerator()
    zoom = lambda x: zoom_gen.apply_transform(x, transform_parameters)
    
    aug_gens = dict()
    aug_gens['train'] = ImageDataGenerator(
            rotation_range=40,
            fill_mode='nearest',
            preprocessing_function=zoom)
    aug_gens['validation'] = ImageDataGenerator(
            rotation_range=40,
            fill_mode='nearest',
            preprocessing_function=zoom)
    aug_gens['test'] = ImageDataGenerator(
            preprocessing_function=zoom)
    
    # get generator per dataset
    ordered_gens = dict()
    kwargs = dict(
        target_size=image_shape,
        batch_size=1,
        class_mode=None,
        shuffle=False
    )
    kwargs.update(flow_kwargs)
    for key, aug_gen in aug_gens.items():
        ordered_gens[key] = aug_gen.flow_from_directory(
            '../data/data/{}'.format(key), **kwargs)
    
    # generate bottleneck labels after augmentation
    labels = dict()
    for key, gen in ordered_gens.items():
        if key == 'test':
            labels[key] = gen.classes
        else:
            labels[key] = np.tile(gen.classes, augment_factor)

    # generate bottlenecks by dataset
    kwargs = dict(
        verbose=1,
        workers=8,
        use_multiprocessing=True,
    )

    bottlenecks = dict()
    for key, gen in ordered_gens.items():
        print('Preparing {} bottlenecks'.format(key))
        bottlenecks[key] = model.predict_generator(
            gen, steps=len(gen) * augment_factor, **kwargs
        )


In [ ]:
class Model:
    @staticmethod
    def get_input_shape(image_shape):
        '''
        Get input shape of conv-nets based on keras backend settings

        Returns
        tuple(n1, n2, n3)
        '''

        if keras.backend.image_data_format() == 'channels_first':
            return (3,) + image_shape 
        else:
            return image_shape + (3,)
        
    def __init__(self, model_func, image_shape, name, codename):
        self.model_func = model_func
        self.image_shape = image_shape
        self.name = name
        self.codename = codename
        self.model = None

    def get_model(self):
        if self.model == None:
            print('loading {} model'.format(self.name))
            self.model = self.model_func(
                weights='imagenet',
                include_top=False,
                input_shape=Model.get_input_shape(self.image_shape))
        return self.model
    
    def free_model(self):
        self.model = None

In [ ]:
models = [
    Model(mobilenet.MobileNet, (224, 224), 'mobilenet', 'MOB'),
    Model(mobilenet_v2.MobileNetV2, (224, 224), 'mobilenetv2', 'MOB2'),
    Model(inception_resnet_v2.InceptionResNetV2, (299, 299), 'inceptionresnetv2', 'INCRES2'),
    Model(inception_v3.InceptionV3, (299, 299), 'inceptionv3', 'INC3'),
    #(densenet.DenseNet, (224, 224), 'densenet'),
    Model(nasnet.NASNet, (224, 224), 'nasnet', 'NAS'),
    Model(resnet50.ResNet50, (224, 224), 'resnet50', 'RES'),
    Model(vgg16.VGG16, (224, 224), 'vgg16', 'VGG16'),
    Model(vgg19.VGG19, (224, 244), 'vgg19', 'VGG19'),
    Model(xception.Xception, (299, 299), 'xception',' XC'),
]

In [ ]:
datasets = ['test', 'train', 'validation']
BOTTLENECK_DIR = 'bottlenecks'
LABEL_DIR = 'labels'


def load_bottlenecks(model: Model, flow_kwargs={}):
    bottleneck_dir = os.path.join(BOTTLENECK_DIR, model.codename)
    label_dir = os.path.join(LABEL_DIR, model.codename)
    
    try:
        bottlenecks = dict()
        labels = dict()
        for dataset in datasets:
            bottlenecks[dataset] = np.load(
                open(os.path.join(bottleneck_dir, '{}.npy'.format(dataset)), 'rb'))
            labels[dataset] = np.load(
                open(os.path.join(label_dir, '{}.npy'.format(dataset)), 'rb'))
        print('loaded existing bottlenecks')
    except FileNotFoundError as e:
        print('generating bottlenecks...')
        bottlenecks, labels = generate_bottlenecks_and_labels(
            model.get_model(), model.image_shape, augment_factor=5, flow_kwargs=flow_kwargs)
        os.makedirs(bottleneck_dir, exist_ok=True)
        for dataset, val in bottlenecks.items():
            np.save(open(os.path.join(bottleneck_dir, '{}.npy'.format(dataset)), 'wb'), val)
        os.makedirs(label_dir, exist_ok=True)
        for dataset, val in labels.items():
            np.save(open(os.path.join(label_dir, '{}.npy'.format(dataset)), 'wb'), val)
            
    return bottlenecks, labels

In [ ]:
def train_model_and_report(model: Model, optimizer, epochs, train_name=None, visualize=False):
    now = datetime.datetime.now().strftime('%y-%m-%d-%H-%M')
    if train_name:
        identifier = '{}_{}'.format(train_name, now)
    else:
        identifier = '{}_{}'.format(model.codename, now)
        
    print('loading bottlenecks')
    bottlenecks, labels = load_bottlenecks(model)

    print('initializing top model for {}'.format(model.name))
    top_model = Sequential()
    top_model.add(Flatten(input_shape=model.get_model().output_shape[1:]))
    top_model.add(Dense(1024, activation='relu'))
    top_model.add(Dropout(0.5))
    '''
    top_model.add(Dense(256, 
                        activation='relu',
                        kernel_initializer='random_uniform',
                        bias_initializer='zeros'))
    top_model.add(Dropout(0.5))
    '''
    top_model.add(Dense(3, activation='softmax'))
    top_model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])

    # one-hot labels
    hot_labels = dict()
    for key, label_array in labels.items():
        hot_labels[key] = to_categorical(label_array, num_classes=3)

    os.makedirs('logs', exist_ok=True)
    tensorboard = TensorBoard(log_dir="logs/{}".format(identifier))
    
    print('training model')
    # train model
    batch_size = 16
    if visualize:
        callbacks=[tensorboard, PlotLossesKeras()]
    else:
        callbacks=[tensorboard]
    top_model.fit(bottlenecks['train'], hot_labels['train'],
                  validation_data=(bottlenecks['validation'], hot_labels['validation']),
                  epochs=epochs,
                  batch_size=batch_size,
                  shuffle=True,
                  callbacks=callbacks)

    print('evaluate model')
    # evaluate model
    results = top_model.evaluate(bottlenecks['test'], hot_labels['test'])

    # save weights for model
    top_model.save_weights('weights/{}.hdf5'.format(identifier))
    
    print(name, results)

In [ ]:
optimizer = optimizers.SGD(lr=1.0e-4, momentum=0.9)
train_model_and_report(models[3], optimizer, 25, train_name=None, visualize=False)

In [ ]:
Model = models[0]
train_name = 'ho'
now = datetime.datetime.now().strftime('%y-%m-%d-%H-%M')
if train_name:
    identifier = '{}_{}'.format(train_name, now)
else:
    identifier = '{}_{}'.format(model.codename, now)

print('loading bottlenecks')
bottlenecks, labels = load_bottlenecks(model)

print('initializing top model')
top_model = Sequential()
top_model.add(Flatten(input_shape=model.get_model().output_shape[1:]))
top_model.add(Dense(1024, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='softmax'))
top_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

# one-hot labels
hot_labels = dict()
for key, label_array in labels.items():
    hot_labels[key] = to_categorical(label_array, num_classes=3)

os.makedirs('logs', exist_ok=True)
tensorboard = TensorBoard(log_dir="logs/{}".format(identifier))

print('training model')
# train model
batch_size = 16
if visualize:
    callbacks=[tensorboard, PlotLossesKeras()]
else:
    callbacks=[tensorboard]
top_model.fit(bottlenecks['train'], hot_labels['train'],
              validation_data=(bottlenecks['validation'], hot_labels['validation']),
              epochs=epochs,
              batch_size=batch_size,
              shuffle=True,
              callbacks=callbacks)

In [ ]:
### STOPPPP

In [ ]:
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

os.makedirs('logs', exist_ok=True)
tensorboard = TensorBoard(log_dir="logs/{}".format(identifier))

# train model
batch_size = 16
if visualize:
    callbacks=[tensorboard, PlotLossesKeras()]
else:
    callbacks=[tensorboard]

if visualize:
    callbacks=[tensorboard, PlotLossesKeras()]
else:
    callbacks=[tensorboard]
    
kwargs = dict(validation_data=(bottlenecks['validation'], hot_labels['validation']),
                  epochs=100,
                  batch_size=16,
                  shuffle=True,
                  callbacks=callbacks)

In [ ]:
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#train_and_eval(mobilenet.MobileNet, (224, 224), 'mobilenet')
for model, shape, name in models:
    train_and_eval(model, shape, name)

# Draft Code...
Two-layer tuning.

In [ ]:

pre_model = models[3].get_model()
# inception resnet v2

In [ ]:
for layer in pre_model.layers[-3:]:
    layer.trainable = True
for layer in pre_model.layers[:-3]:
    layer.trainable = False
for layer in pre_model.layers[-3:]:
    layer.trainable = False

In [ ]:
new_model = Sequential()
new_model.add(pre_model)

In [ ]:
top_model = Sequential()
top_model.add(Flatten(input_shape=pre_model.output_shape[1:]))
top_model.add(Dense(1024, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='softmax'))

new_model.add(top_model)

In [ ]:
top_model.load_weights('bottleneck_fc_model.h5')

In [ ]:
new_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  #optimizer='rmsprop',
                  metrics=['accuracy'])

In [ ]:
batch_size = 16
IMAGE_SHAPE = models[2].image_shape

transform_parameters = {
    'zx': 0.6,
    'zy': 0.6,
}
zoom_gen = ImageDataGenerator()
zoom = lambda x: zoom_gen.apply_transform(x, transform_parameters)
gen = ImageDataGenerator(
        preprocessing_function=zoom)
aug_gen = ImageDataGenerator(
        rotation_range=40,
        fill_mode='nearest',
        preprocessing_function=zoom)
aug_gen2 = ImageDataGenerator(
        rotation_range=40,
        fill_mode='nearest',
        preprocessing_function=zoom)

train_image_generator = aug_gen.flow_from_directory(
    '../data/data/train',
    target_size=(IMAGE_SHAPE),
    batch_size=batch_size)

validation_image_generator = aug_gen2.flow_from_directory(
    '../data/data/validation',
    target_size=(IMAGE_SHAPE),
    batch_size=batch_size)

test_image_generator = gen.flow_from_directory(
    '../data/data/test',
    target_size=(IMAGE_SHAPE),
    batch_size=batch_size,
    shuffle=False)

In [ ]:
epoch_count = 50
callbacks=[PlotLossesKeras()]
new_model.fit_generator(
    train_image_generator,
    epochs=epoch_count,
    validation_data=validation_image_generator,
    workers=8,
    use_multiprocessing=True,
    callbacks=callbacks)

In [ ]:
new_model.evaluate_generator(test_image_generator)

In [ ]:
new_model.predict_generator(train_image_generator)